## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-11-08-22-37-55 +0000,nypost,Bill Maher’s Cheney death joke draws groans,https://nypost.com/2025/11/08/us-news/bill-mah...
1,2025-11-08-22-28-35 +0000,nypost,‘Teacher of the Year’ placed on leave after al...,https://nypost.com/2025/11/08/us-news/teacher-...
2,2025-11-08-22-28-13 +0000,nyt,Trump Gives Hungary a Reprieve on Sanctions Af...,https://www.nytimes.com/2025/11/08/us/politics...
3,2025-11-08-22-25-49 +0000,bbc,UK seeks Danish inspiration to shake up immigr...,https://www.bbc.com/news/articles/cm2lknr2d3go...
4,2025-11-08-22-24-59 +0000,nyt,"Gavin Newsom, Eyeing 2028, Tries to Mess With ...",https://www.nytimes.com/2025/11/08/us/politics...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2364/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
19,trump,34
134,shutdown,11
31,up,11
130,flight,9
405,mamdani,9


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
173,2025-11-08-02-00-00 +0000,wsj,Republicans in Washington got a clear takeaway...,https://www.wsj.com/politics/policy/trump-pric...,83
50,2025-11-08-17-30-00 +0000,wsj,"Middle-class voters, many of them frustrated w...",https://www.wsj.com/politics/elections/democra...,76
164,2025-11-08-03-45-03 +0000,nyt,Trump Administration Appeals to Supreme Court ...,https://www.nytimes.com/2025/11/07/us/politics...,74
166,2025-11-08-02-59-00 +0000,wsj,Supreme Court Pauses Order for Trump Administr...,https://www.wsj.com/us-news/law/supreme-court-...,73
187,2025-11-08-00-07-13 +0000,latimes,Supreme Court blocks order for Trump administr...,https://www.latimes.com/california/story/2025-...,69


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
173,83,2025-11-08-02-00-00 +0000,wsj,Republicans in Washington got a clear takeaway...,https://www.wsj.com/politics/policy/trump-pric...
61,52,2025-11-08-17-17-26 +0000,nypost,New Yorkers flock to food banks as government ...,https://nypost.com/2025/11/08/us-news/trump-ur...
79,47,2025-11-08-14-29-08 +0000,cbc,"UPS, FedEx ground MD-11 air cargo fleets after...",https://www.cbc.ca/news/world/ups-fedex-ground...
199,44,2025-11-07-22-54-46 +0000,nypost,Top NYC realtor fired for comparing Zohran Mam...,https://nypost.com/2025/11/07/business/top-nyc...
164,32,2025-11-08-03-45-03 +0000,nyt,Trump Administration Appeals to Supreme Court ...,https://www.nytimes.com/2025/11/07/us/politics...
27,31,2025-11-08-20-19-55 +0000,nypost,Singer claims transgender woman — who beat ex-...,https://nypost.com/2025/11/08/us-news/transgen...
55,29,2025-11-08-17-23-43 +0000,nypost,"Andrew Mountbatten Windsor, Fergie being probe...",https://nypost.com/2025/11/08/world-news/princ...
176,29,2025-11-08-01-18-00 +0000,wsj,Senate Minority Leader Chuck Schumer proposed ...,https://www.wsj.com/politics/policy/emboldened...
195,28,2025-11-07-23-36-31 +0000,wapo,Flight cancellations hit major airports on fir...,https://www.washingtonpost.com/transportation/...
97,28,2025-11-08-12-38-25 +0000,nyt,Pfizer to Acquire Obesity Drug Start-Up Metser...,https://www.nytimes.com/2025/11/08/business/me...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
